In [2]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2017-10-22T11:55:31+02:00

CPython 3.6.1
IPython 5.3.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-37-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Los clasificadores Naive Bayes (Naive Bayes Classifier - NBC) se usan , como su nombre indica, para problemas de clasificación, y en concreto se pueden aplicar para texto. 

En este ejemplo vamos a implementar un modelo NBC a un dataset de un portal de Noticias muy famoso en España. Cada usuario comparte un link a una noticia y le puede asignar una categoría.

In [3]:
import pandas as pd

noticias = pd.read_csv("data/noticias.csv")

noticias.head()

,descripcion,categoria
0,"Aunque parezca mentira, las emisiones de dióxi...",cultura
1,Hubo un proyecto impulsado por la Unión Europe...,cultura
2,China ha confirmado la conclusión con éxito de...,tecnología
3,"En su fructífera carrera como humorista, actor...",cultura
4,Tras dos años de negociación entre la instituc...,cultura


La variable objetivo es `categoria` y la variable independiente es `descripcion` que contiene la descripcion de la noticia

In [4]:
noticias.categoria.value_counts()

cultura       9001
tecnología    4198
ocio          3296
Name: categoria, dtype: int64

Vemos que hay noticias de 3 tipos de categorías distintas.

Los clasificadores Naive Bayes esperan como input un vector, así que para poder entrenarlos tenemos que vectorizar el texto. Para ello una buena opción es usar vectorización Tf-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
TfidfVectorizer?

Vamos a modificar nuestro vectorizador añadiendole dos parámetros:

** Eliminar Stopwords**

Stopwords (palabras vacías) son palabras que no tienen ningún contenido semántico. Por ejemplo, en la frase `el perro ladra` el artículo `el` no aporta ningún valor a la frase.


Me he descargado una lista de stopwords de castellano de este [repositorio en Github](https://github.com/stopwords-iso/stopwords-es).

In [8]:
import json


with open("data/stopwords-es.json") as fname:
    stopwords_es = json.load(fname)

In [9]:
stopwords_es[:30]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '_',
 'a',
 'actualmente',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'adrede',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún']

**Eliminar acentos**

También vamos a eliminar los acentos de las palabras, ésto tiene una ventaja, y es que si en el conjunto de datos no tenemos confianza en la calidad de los escritores, al eliminar los acentos evitamos que si un escritor no usa acentos no considere sus palabras como palabras distintas.

En castellano, esto tiene un problema, y es que hay palabras con significado distinto que sólo se diferencian por la existencia de una tilde (se llaman palabras con [acento diacrítico](https://es.wikipedia.org/wiki/Acento_diacr%C3%ADtico) (por ejemplo, `de y dé`). Asumimos pues que el impacto de estas palabras no es muy alto.

In [10]:
vectorizador = TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es)

In [12]:
noticias.shape

(16495, 2)

In [11]:
vectorizador.fit_transform(noticias.descripcion)

<16495x59969 sparse matrix of type '<class 'numpy.float64'>'
	with 397709 stored elements in Compressed Sparse Row format>

Dicha matriz nos indica que tenemos 16495 artículos que tienen 59969 palabras distintas (sin contar acentos o stopwords)

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

Dado que los vectorizadores devuelven una matriz sparse (escasa) creamos un transformador que las convierta a densas

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from scipy.sparse import issparse


# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [15]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

Scikit-learn tiene tres implementaciones del clasificador [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html), `GaussianNB, BernoulliNB y MultinomialNB`, y cada una se diferencia por como calcula las probabilidades de que cada elemento aparezca en los datos.

[GaussianNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) asume que los datos siguen una distribución Gausiana

In [16]:
pipeline_gaussiano = make_pipeline(
    vectorizador,
    DenseTransformer(),
    GaussianNB()
)

In [18]:
pipeline_gaussiano.fit(X=noticias.descripcion, y=noticias.categoria)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i..., ('densetransformer', DenseTransformer(return_copy=True)), ('gaussiannb', GaussianNB(priors=None))])

In [19]:
pipeline_gaussiano.predict(noticias.descripcion)

array(['cultura', 'cultura', 'tecnología', ..., 'cultura', 'tecnología',
       'ocio'], 
      dtype='<U10')

In [20]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring="f1")

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [21]:
from sklearn.metrics import f1_score


In [24]:
f1_score??

Vemos que la validación cruzada con la puntuación F1 nos da un error `ValueError: Target is multiclass but average='binary'. Please choose another average setting.`, esto es por que las medidas de clasificación tienen distintas maneras de calcularse en funcion de si es un caso de clasificación binaria (el parámetro por defecto) o clasificación multiclase. 

En concreto para el caso de la puntuación F1, nos permite los siguientes tipos de cálculos.

- **binary**: Devuelve la puntuación para la clase especificada en el argumento `pos_label`. Solo se puede aplicar en clasificación binaria. Este es el caso que vimos en el apartado de medidas de evaluación de modelos de clasificación.

- **micro**: Cuenta todos el número total de Verdaderos positivos (TP), Falsos Negativos (FN) y Falsos Positivos (FP) y calcula una precisión y sensibilidad total y obtiene el F1. Es mejor cuando hay clases no balanceadas (muchos casos más de una clase que de las demás).

- **macro**: Calcula la precisión y sensibilidad media de cada clase, hace su media aritmética y calcula el parámetro F1.

- **weighted**: Calcula la precisión y sensibilidad media de cada clase, hace su media ponderada por el número de observaciones de cada clase y calcula el parámetro F1.

- **samples**

In [25]:
from sklearn.metrics import f1_score


def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

In [27]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

array([ 0.63854545,  0.65205529,  0.63361834])

Como hemos visto antes, el vectorizador TF-IDF coge por defecto todas las palabras (removiendo palabras sin aporte de información). Podemos restringir la cantidad de palabras que considera con el parámetro `max_features`.

In [31]:
pipeline_gaussiano = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=1000),
    DenseTransformer(),
    GaussianNB()
)

In [32]:
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

array([ 0.56636364,  0.58766824,  0.55848645])

Vemos que el modelo funciona de igual forma ahora pese a que los datos de entrenamiento son bastante más pequeños (una matriz de dimensión **16495x500** en vez de una de dimensión **16495x59969**

Las dos implementaciones de clasificadores NB más utilizadas para clasificación de texto son [MultinomialNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) (que asume que la distribución de probabilidades de las palabras en el conjunto de datos sigue una distribución multinomial y [BernouilliNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB) , que asume que siguen una distribución de Bernouilli multivariable (donde la existencia de cada palabra se considera que es una variable binaria distinta).

Según la documentación, el clasificador MultinomialNB funciona bien con vectores TFIDF

In [33]:
pipeline_multinomial = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=500),
    DenseTransformer(),
    MultinomialNB(),
)

In [35]:
cross_val_score(pipeline_multinomial, noticias.descripcion, noticias.categoria,
                scoring=f1_multietiqueta)

array([ 0.68527273,  0.6751546 ,  0.67691468])

Vemos que efectivamente, `MultinomialNB` parece funcionar mejor que `GaussianNB` para vectores TF-IDF

Para el clasificador [BernouilliNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB) se necesita tener los vectores de palabras cono vectores binarios (1 si la palabra existe o 0 si no), así que en vez de usar TfidfVectorizer en este caso usaremos el CountVectorizer pasandole el parámetro `binary=True` para que devuelva 1 ó 0 en vez del número de veces que aparece la palabra en la frase.

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_count = CountVectorizer(stop_words=stopwords_es, binary=True, 
                                     strip_accents="unicode", max_features=1000)

Una funcionalidad interesante del CountVectorizer es que nos permite ver cuantas veces aparece cada palabra en el corpus.

In [38]:
vectorizador_count.fit(noticias.descripcion)
vectorizador_count.vocabulary_

{'tierra': 929,
 'verde': 969,
 '30': 25,
 'anos': 76,
 'epoca': 338,
 'principal': 766,
 'estudio': 368,
 'internacional': 501,
 'publicado': 795,
 'pasar': 708,
 'proyecto': 788,
 'union': 950,
 'europea': 372,
 'metodo': 599,
 'lengua': 528,
 'espanol': 350,
 'sistema': 882,
 'espanoles': 352,
 'puedan': 801,
 'entender': 333,
 'china': 166,
 'confirmado': 204,
 'exito': 380,
 'forma': 404,
 'totalmente': 935,
 'robot': 844,
 'logro': 553,
 'personal': 725,
 'humano': 463,
 'carrera': 156,
 'actor': 48,
 'director': 288,
 'escritor': 345,
 'decenas': 255,
 'peliculas': 711,
 'importantes': 477,
 'historia': 454,
 'cine': 174,
 'humor': 465,
 'mayoria': 585,
 'encuentran': 327,
 'publico': 798,
 'online': 677,
 'mejores': 591,
 'museo': 630,
 'finalmente': 399,
 'siglo': 874,
 'musica': 631,
 'texto': 926,
 'creado': 233,
 'nombre': 651,
 'obra': 663,
 'video': 976,
 'hombre': 457,
 'comida': 190,
 'muestra': 624,
 'problema': 772,
 'usando': 953,
 'google': 433,
 'dando': 248,
 'mun

In [39]:
pipeline_bernouilli = make_pipeline(
    vectorizador_count,
    DenseTransformer(),
    BernoulliNB(),
)

In [40]:
cross_val_score(pipeline_bernouilli, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta)

array([ 0.67636364,  0.68297563,  0.67746043])